In [25]:
import os
imdb_dir = '/home/akkasi/Desktop/aclImdb/aclImdb'
train_dir = os.path.join(imdb_dir,'train')

In [23]:
labels = []
texts = []

# Data preparation (Reading + Tokenization)

In [57]:
for label_type in ['neg','pos']:
    dir_name = os.path.join(train_dir,label_type)
    for fname in sorted(os.listdir(dir_name)):
        if fname[-4:] == '.txt':
            f = open(os.path.join(dir_name,fname))
            texts.append(f.read())
            f.close()
            if label_type == 'neg':
                labels.append(0)
            else:
                labels.append(1)
                
        

In [58]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np

In [75]:
maxlen =100
training_samples = 200
validation_samples = 10000
max_words= 10000
tokenizer =Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
len(sequences[125])

205

In [60]:
word_index = tokenizer.word_index

In [82]:
data = pad_sequences(sequences,maxlen=maxlen)


In [62]:
labels =np.asarray(labels)

In [63]:
indicies = np.arange(data.shape[0])
np.random.shuffle(indicies)
data = data[indicies]
labels = labels[indicies]

In [64]:
x_train = data[:training_samples]
y_train = labels[:training_samples]
x_val = data[training_samples:training_samples+validation_samples]
y_val = labels[training_samples:training_samples+validation_samples]


In [65]:
glove_dir = '/home/akkasi/Desktop/glove.6B'
embeddings_index = {}
f = open(os.path.join(glove_dir,'glove.6B.100d.txt'))
for line in f:
    values =line.split()
    word = values[0]
    coefs = np.asarray(values[1:],dtype='float32')
    embeddings_index[word] = coefs
f.close()

In [66]:
embedding_dim = 100
embedding_matrix = np.zeros((max_words,embedding_dim))
for word, i in word_index.items():
    if i < max_words:
        embedding_vector =embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector

# DEFINING A MODEL

In [67]:
from keras.models import Sequential
from keras.layers import Dense,Flatten,Embedding

In [68]:
model = Sequential()
model.add(Embedding(max_words,embedding_dim,input_length=maxlen))
model.add(Flatten())
model.add(Dense(32,activation='relu'))
model.add(Dense(1,activation='sigmoid'))


In [69]:
model.layers[0].set_weights([embedding_matrix])
model.layers[0].trainable = False
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 100, 100)          1000000   
_________________________________________________________________
flatten_5 (Flatten)          (None, 10000)             0         
_________________________________________________________________
dense_7 (Dense)              (None, 32)                320032    
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 33        
Total params: 1,320,065
Trainable params: 320,065
Non-trainable params: 1,000,000
_________________________________________________________________


# Training And Evaluating The Model

In [70]:
model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics = ['acc'])
history = model.fit(x_train,y_train,epochs=10,batch_size=32,validation_data=(x_val,y_val))
model.save_weights('pre_trained_glove_model.h5')

Train on 200 samples, validate on 10000 samples
Epoch 1/10
200/200 [==============================] - 1s 3ms/step - loss: 2.5744 - acc: 0.4850 - val_loss: 2.0270 - val_acc: 0.4972
Epoch 2/10
200/200 [==============================] - 1s 3ms/step - loss: 0.6595 - acc: 0.7000 - val_loss: 1.5628 - val_acc: 0.5038
Epoch 3/10
200/200 [==============================] - 1s 3ms/step - loss: 0.5039 - acc: 0.7400 - val_loss: 0.8239 - val_acc: 0.5200
Epoch 4/10
200/200 [==============================] - 0s 2ms/step - loss: 0.1891 - acc: 0.9800 - val_loss: 1.2621 - val_acc: 0.4980
Epoch 5/10
200/200 [==============================] - 1s 3ms/step - loss: 0.2573 - acc: 0.8650 - val_loss: 0.9111 - val_acc: 0.5339
Epoch 6/10
200/200 [==============================] - 0s 2ms/step - loss: 0.1082 - acc: 0.9800 - val_loss: 0.7617 - val_acc: 0.5716
Epoch 7/10
200/200 [==============================] - 1s 3ms/step - loss: 0.0739 - acc: 0.9850 - val_loss: 0.9858 - val_acc: 0.5207
Epoch 8/10
200/200 [========

# Generalization 

In [71]:
test_dir = os.path.join(imdb_dir,'test')
labels= []
texts =[]
for label_type in ['neg','pos']:
    dir_name = os.path.join(test_dir,label_type)
    for fname in sorted(os.listdir(dir_name)):
        if fname[-4:] == '.txt':
            f = open(os.path.join(dir_name,fname))
            texts.append(f.read())
            f.close()
            if label_type =='neg':
                labels.append(0)
            else:
                labels.append(1)
sequences = tokenizer.texts_to_sequences(texts)
x_test = pad_sequences(sequences,maxlen=maxlen)
y_test = np.asanyarray(labels)


model.load_weights('pre_trained_glove_model.h5')
model.evaluate(x_test,y_test)

25000/25000 [==============================] - 1s 50us/step


[4.001258310619463, 0.50016]